In [1]:
#!pip install pymysql

In [2]:
import os
import pymysql
import warnings
import string
import csv
import sys
warnings.filterwarnings('ignore')

In [3]:
import mysql.connector as connector
from datetime import date

In [4]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

In [5]:
global conn, cursor
conn = pymysql.connect(
    host=host,port=int(3306),
    user="root",
    passwd="Prdp@1996",
    database="bill_system",
    charset='utf8mb4')

cursor = conn.cursor()

In [6]:
def clear():                            #To clear the screen.
    for _ in range(5):
        print()

def login():
    while True:
        clear()
        uname=input('enter your id:')
        upass=input('enter your password')
        cursor.execute('select *from login where name="{}"and pwd="{}"'.format(uname,upass))   #cursor is global variable
        cursor.fetchall()                                                            #full the record
        rows=cursor.rowcount                                                      #count the row of record
        if rows!=1:
            print('invalid login details....try again')
        else:
            print('you are elegible for operating this system...')
            print('\n\n')
            print('Press any key to continue.....')
            break

def last_bill_no():
    cursor.execute('select max(bill_id) from bills')                      #find the max bill number
    record = cursor.fetchone()
    return record



def find_item(no):
    cursor.execute('select * from items where id ={}'.format(no))         #getting 
    record = cursor.fetchone()
    return record

In [7]:
def add_item():                                                                   #this fuction adding the items into database 
    clear()
    print('Add New Item - ')
    print('-'*100)
    item_name = input('Enter new Item Name :')                                     #enter the new item
    item_price = input('Enter Item Price :')                                       #enter the item price
    sql = 'select * from items where item_name like "%{}%"'.format(item_name)      #to check the entered item wether present in previous table(DB) 
    cursor.execute(sql)                                                            #cursor going to excute the above sql query
    record=cursor.fetchone()                                                       #we will store the data in record and fetchone() will fetch one row
    if record==None:                                                                #comparing the store value in record with None
        sql = 'insert into items(item_name,price) values("{}",{});'.format(item_name,item_price)    # inserting values into table(db) by sql query using farmat method 
        cursor.execute(sql)                                                         
        print('\n\nNew Item added successfully.....\nPress any key to continue....')
    else:
        print('\n\nItem Name already Exist.....\nPress any key to continue....')
    wait= input()

In [8]:
#   function name       : modify_item
#   purpose             : change item details in items table
def modify_item():                                             
    clear()
    print('Modify Item Details - ')            
    print('-'*100)
    item_id = input('Enter Item ID :')                          #enter the item id 
    item_name = input('Enter new Item Name :')                  #enter the item name nd price
    item_price = input('Enter Item Price :')                    #UPDATE table_name  SET column1 = value1, column2 = value2, ...   WHERE condition;
    sql = 'update items set item_name = "{}", price ={} where id={}'.format(item_name,item_price,item_id)
    cursor.execute(sql)
    print('\n\nRecord Updated Successfully............')

In [9]:
#   function name           : item_list
#   purpose                 : To display all the items in items tables
def item_list():
    clear()
    sql="select * from items"
    cursor.execute(sql)
    records = cursor.fetchall()
    for row in records:
        print(row)
    print('\nPress any key to continue.....')         #the program will wait indefinitely for the user input. 
    wait = input()                                   #Once the user provides the input data and presses the enter key, the program will start executing the next statements

In [10]:
#   function name       : billing
#   purpose             : To generate bills 
def billing():
    clear()
    items =[]                        #store the items
    bill_no = last_bill_no()         # checking last "bill number" & find the last bill number from databsae from def last_bill_no (line 19)
    if bill_no[0]==None:             #bill no is null then assign bill num 1  #None 
        bill_no =1
    else:
        bill_no = bill_no[0]+1                     # if we have last bill number then will generate new bill number by adding =1

    name = input('Enter customer Name :')          #collecting the customer name & mobile number
    phone = input('Enter Phone No :')       
    today = date.today()                           #getting present date by today() fuction
    while True:                                    #WHY while loop? -> execution is done in the while loop until the statement in the program is proved wrong.
        no = int(input('Enter item No (0 to stop) :'))   # enter the item number & it will stored
        if no <=0:                                       #if value is zero or less than zero then loop will break and going to generate the bills
            break
        else:
            item = find_item(no)                         #check wether item wether we have in our table(db) from def find_item(no):
            if item==None:
                print('Item Not found  ')
            else:
                qty = int(input('Enter Item Qty :'))      #enter the quantity of items
                item = list(item)                         #making the list the item
                item.append(qty)                          #
                items.append(item)

    clear()
    print('                     Digikull Shopping mall              ')
    print('                     j- Block IT City, PYsQl     ')
    print('                     Phone: 9988776655, Email: digikull@shopping.com ')
    print('Bill No :{}        Date :{}'.format(bill_no,today))
    print('-'*100)
    print('Customer Name :{}          Phone No :{}'.format(name,phone))
    print('-'*100)
    print('Item Id     Item Name                Price            Qty         Amount ')
    print('-'*100)
    total =0                                                    # 
    for item in items:                                          #
        print('{:<10d} {:25s} {:.2f} {:>10d}          {:>.2f} \
            '.format(item[0],item[1],item[2],item[3],item[2]*item[3]))
        total = total +item[2]*item[3]
    print('-'*100)
    print('Total Payable amount : {}'.format(total))
    print('\nPress any key to continue........')
    #insert data into tables
    sql ='insert into bills(name,phone,bill_date) values("{}","{}","{}");'.format(name,phone,today)
    cursor.execute(sql)    
    for item in items:
        sql='insert into transaction(item_id,qty,bill_id) values({},{},{});'.format(item[0],item[3],bill_no)
        cursor.execute(sql)
    wait= input()


In [11]:
#   function      : Date_wise_sell
#   purpose       : Create a report on date wise sell or sell between two dates 
def date_wise_sell():
    clear()
    print('Sell Between Two Dates -- ')
    print('-'*100)
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End Date (yyyy-mm-dd) :')
    sql = 'select * from bills where bill_date between "{}" and "{}"'.format(start_date,end_date)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Bill No       Customer Name         Phone No         Bill Date')
    print('-'*100)
    for row in records:
        print('{:10d} {:30s} {:20s} {}'.format(row[0],row[1],row[2],row[3]))
    print('-'*100)
    print('\n\nPress any key to continue....')
    wait= input()


In [12]:
# function name        : bill information
# purpose               : display details of any bill
def bill_information():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    cursor.execute(sql) 
    records = cursor.fetchall()
    n = cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:<.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()   


In [13]:
#  function name    : amount_collected
#  purpose          : Function to display amount collected between two dates
def amount_collected():
    clear()
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End   Date (yyyy-mm-dd) :')
    clear()
    print('Amount collected between: {} and {}'.format(start_date,end_date))
    print('-'*100)
    sql = 'select sum(t.qty*i.price) from bills b,transaction t,items i \
           where b.bill_date between "{}" AND "{}" AND b.bill_id = t.bill_id AND \
           t.item_id = i.id;'.format(start_date,end_date)
    cursor.execute(sql)
    result = cursor.fetchone()
    print(result)
    print('\nPress any key to continue.....')
    wait= input()


In [14]:
def search_item():
    clear()
    item_name =input('Enter Item Name :')
    sql ='select * from items where item_name like "%{}%";'.format(item_name)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Item Names start with :',item_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s}'.format('Item ID','Item Name','Item Price'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:.2f}'.format(row[0],row[1],row[2]))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()


In [15]:
def search_customer():
    clear()
    cust_name =input('Enter customer Name :')
    sql ='select * from bills where name like "%{}%";'.format(cust_name)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Customer Names started with :',cust_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s} {:20s}'.format('Bill No','Customer Name','Phone No','Bill Date'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:20s} {:20s}'.format(row[0],row[1],row[2],str(row[3])))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()


In [16]:
# function name      : search_bill
# purpose            : function to find out bill information
def search_bill():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    cursor.execute(sql) 
    records = cursor.fetchall()
    n = cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:<.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()   


In [17]:
#  function name    : search_menu
#  purpose          : Display search menu on the screen
def search_menu():
    while True:
        clear()
        print('      S E A R C H    M E N U ')
        print('-'*100)
        print('1.   Item Name')
        print('2.   Customer information')
        print('3.   Bill information')
        print('4.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..4): '))
        if choice==1:
            search_item()
        if choice==2:
            search_customer()
        if choice==3:
            search_bill()
        if choice==4:
            break


In [18]:
#  function name    : report_menu
#  purpose          : Display report menu on the screen
def report_menu():
    while True:
        clear()
        print('   R E P O R T   M E N U ')
        print('-'*100)
        print('1.   Item List')
        print('2.   Sell Between Dates')
        print('3.   Bill information')
        print('4.   Amount collected')
        print('5.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..5): '))
        if choice==1:
            item_list()
        if choice==2:
            date_wise_sell()
        if choice==3:
            bill_information()
        if choice==4:
            amount_collected()
        if choice==5:
            break


In [ ]:
def main_menu():
    while True:
        clear()
        print('      M A I N   M E N U')
        print('-'*100)
        print('1.   Add New Item')
        print('2.   Modify Existing Item')
        print('3.   Billing')
        print('4.   Search Menu')
        print('5.   Report Menu')
        print('6.   Exit')
        choice = int(input('\n\nEnter your choice (1..6): '))
        if choice==1:
            add_item()
        if choice==2:
            modify_item()
        if choice==3:
            billing()
        if choice==4:
            search_menu()
        if choice==5:
            report_menu()
        if choice==6:
            break


if __name__=="__main__":
    clear()
    login()
    main_menu()











enter your id:100
enter your password1
invalid login details....try again





enter your id:Pradeep
enter your passwordP12345
you are elegible for operating this system...



Press any key to continue.....





      M A I N   M E N U
----------------------------------------------------------------------------------------------------
1.   Add New Item
2.   Modify Existing Item
3.   Billing
4.   Search Menu
5.   Report Menu
6.   Exit


Enter your choice (1..6): 1





Add New Item - 
----------------------------------------------------------------------------------------------------
Enter new Item Name :Laptop
Enter Item Price :25000


New Item added successfully.....
Press any key to continue....
